In [1]:
import numpy as np
import pandas as pd
import math

df = pd.read_csv("./input/cantine-2011-2019.csv", sep=";")
df.head()

,Date,Plat
0,2011-01-04,Crêpe au fromage
1,2011-01-04,haricots beurre / haricots plats
2,2011-01-11,Tomme bio
3,2011-01-12,Jambon fumé
4,2011-01-12,St Nectaire


# Exclusion des informations non alimentaire (fichier exclusion)

In [2]:
exclure = pd.read_csv("./input/exclusion.csv", sep=",", header=None)[0].values.tolist()

In [3]:
exclure

['repas grève',
 'sapin de noel',
 'PA lundi',
 'PA jeudi',
 'Journée internationale sans viande',
 'reprise scolaire',
 'REPAS POUR LA PLANETE',
 'MANGER BOUGER',
 'GRANDS MENAGES',
 'greves',
 'reprise scolaire',
 'Chandeleur',
 'Ascension',
 'repas greves',
 'Pentecôte',
 'PA de mardi',
 'PA mardi',
 '-',
 'menu depannage',
 'menu de depannage',
 'repas de Noël cl',
 'SANS CLASSE',
 'lundi de pâques',
 'pesée)',
 'MERCREDI SCOLAIRE',
 'menu greve',
 'Service minimum',
 'MELODIE DES METS LOCAUX',
 'SCOLAIRE',
 'REPAS DE NOEL',
 'Grève',
 'SERVICE MINIMUM',
 'Menu depannage',
 'Sapin de noel',
 'repas de depannage',
 'Repas festif??',
 'repas grêve',
 'NOUVEL AN CHINOIS']

In [4]:
df[df['Plat'].isin(exclure)].size

248

In [5]:
df2 = df[~df['Plat'].isin(exclure)]

In [6]:
len(df2.index)

9866


# Import fichier aliments


In [7]:
aliments = pd.read_csv("./input/aliments.csv", sep=",")
aliments

,categorie,aliment,à ajouter dans
0,viande rouge,boeuf,NaN
1,viande rouge,viande hache surgelée,NaN
2,viande rouge,Rosette bio°,NaN
3,viande rouge,bœuf,NaN
4,viande rouge,agneau,NaN
...,...,...,...
776,autre protéine,omellette,NaN
777,legume,legume,NaN
778,fruit,fruit,NaN
779,sucrerie,sucrerie,NaN


In [8]:
dfAlimentsGroup = aliments.groupby('categorie').agg({'aliment':lambda x: tuple(x)}).reset_index()
dfAlimentsGroup

,categorie,aliment
0,autre protéine,"(Galettes végétale, Galette de soja à l'indien..."
1,feculent,"(Raviolis, semoule, Pâtes, Tagliatelles, coqui..."
2,fruit,"(abricot, airelle, alkékenge, amande, amélanch..."
3,laitage,"(lait, chèvre, camembert, A Filetta, Abbaye de..."
4,legume,"(Ail, Artichaut, Asperge, Aubergine, Avocat, B..."
5,plat préparé,"(hachis parmentier, parmentier de poisson, ome..."
6,poisson fruit de mer,"(poisson, sardine, saumon, anchois, carpe, dor..."
7,produit industriel,"(Poéléé tipiak, Ratatouille, Hachis parmentier..."
8,sauce,"(sauce,)"
9,sucrerie,"(sucrerie,)"


In [9]:
dfAlimentsGroup['regex'] = dfAlimentsGroup.apply(
    lambda row: '|'.join(row['aliment'])
    , axis = 1)

# Tag des plats par aliments

In [10]:
def flagCategorie(df, categorie, alimentRegex):
  df[categorie] = df["Plat"].str.lower().replace("é", "e", regex=True).replace("è", "e", regex=True).str.contains(alimentRegex.lower().replace("é", "e").replace("è", "e"), case=False)

In [11]:
for index, row in dfAlimentsGroup.iterrows():
    flagCategorie(df2, row['categorie'], row['regex'])

<ipython-input-10-e5bfa393f7d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorie] = df["Plat"].str.lower().replace("é", "e", regex=True).replace("è", "e", regex=True).str.contains(alimentRegex.lower().replace("é", "e").replace("è", "e"), case=False)
/home/edith/.local/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [12]:
len(df2.index)

9866

In [13]:
df2.head(3)

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,sauce,sucrerie,viande blanche,viande rouge
0,2011-01-04,Crêpe au fromage,False,False,False,True,False,True,False,False,False,False,False,False
1,2011-01-04,haricots beurre / haricots plats,False,True,False,False,True,False,False,False,False,False,False,False
2,2011-01-11,Tomme bio,False,False,False,True,False,False,False,False,False,False,False,False


## Correction Pâtes et paté
Les Pâtes sont taguées en plat industirle via l'entrée paté

In [14]:
for index, row in df2.iterrows():
  if (('Pâtes'in row['Plat']) & (row['produit industriel'] == True)):
    df2.loc[index,'produit industriel'] = False

/home/edith/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


# Cas spécifique Plat préparé / Plat industriel
Ici, on veut appliquer plusieurs aliments à un plat selon sa composition hypothétique

In [15]:
composition = aliments[
                       ((aliments['categorie'] == "plat préparé") | (aliments['categorie'] == "produit industriel"))
                       & ((aliments['à ajouter dans']))
                       ]

In [16]:
composition['liste-ajout'] = composition['à ajouter dans'].apply(lambda x: x.split(','))

<ipython-input-16-ae7dac65de5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  composition['liste-ajout'] = composition['à ajouter dans'].apply(lambda x: x.split(','))


In [17]:
composition.head()

,categorie,aliment,à ajouter dans,liste-ajout
617,plat préparé,hachis parmentier,"viande rouge,feculent","[viande rouge, feculent]"
618,plat préparé,parmentier de poisson,"poisson fruit de mer,feculent","[poisson fruit de mer, feculent]"
619,plat préparé,omelette,autre protéine,[autre protéine]
620,plat préparé,Omelettte,autre protéine,[autre protéine]
621,plat préparé,Potage,legume,[legume]


In [18]:
# list of aliment we want to change the composition
alimentList = composition['aliment'].tolist()

In [19]:
def addComposition(df):
  for i in df.index:
    plat = df.at[i, "Plat"]
    if (plat in alimentList):
      # get the corresponding list
      categorieList = composition.loc[composition['aliment'] == plat]['liste-ajout'].values[0]
      # tag the ingredient
      for category in categorieList:
        df.at[i, category] = True

In [20]:
addComposition(df2)

# Résultat des traitement

In [21]:
df2[df2['Plat'] == 'Lasagnes'].head(3)

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,sauce,sucrerie,viande blanche,viande rouge
421,2014-02-24,Lasagnes,False,True,False,False,True,True,False,False,False,False,False,True
789,2016-06-23,Lasagnes,False,True,False,False,True,True,False,False,False,False,False,True
1436,2011-11-03,Lasagnes,False,True,False,False,True,True,False,False,False,False,False,True


In [22]:
df2[df2["Date"] == '2019-05-22']

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,sauce,sucrerie,viande blanche,viande rouge
2464,2019-05-22,Radis,False,False,False,False,True,False,False,False,False,False,False,False
2465,2019-05-22,Sauce bolognaise végétale,False,False,False,False,True,True,False,True,True,False,False,False
2466,2019-05-22,Gruyère râpé,False,False,False,True,False,False,False,False,False,False,False,False
3643,2019-05-22,Île flottante,False,False,False,True,False,True,False,False,False,False,False,False
9887,2019-05-22,Pâtes bio et gruyère râpé,False,True,False,True,False,False,False,False,False,False,True,False


# % couverture de la détection des aliments

In [23]:
import functools

In [24]:
categories = dfAlimentsGroup['categorie'].tolist()
categories = [x for x in categories if x != "plat préparé" and x != "produit industriel"]
categories

['autre protéine',
 'feculent',
 'fruit',
 'laitage',
 'legume',
 'poisson fruit de mer',
 'sauce',
 'sucrerie',
 'viande blanche',
 'viande rouge']

In [25]:
columns = []
for cat in categories:
  columns.append(df2[cat])

In [26]:
df2['status'] = functools.reduce(lambda x,y: x | y, columns)

<ipython-input-26-d3a5decb7e3e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['status'] = functools.reduce(lambda x,y: x | y, columns)


In [27]:
### Pourcentage de plats non couvert

In [28]:
with_no_status = df2[(df2["status"] == False)]

len(with_no_status.index) / len(df2.index) * 100

4.490168254611798

In [29]:
df2[(df2["status"] == False)].head(20)

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,sauce,sucrerie,viande blanche,viande rouge,status
14,2011-01-21,beurre blanc,False,False,False,False,False,False,False,False,False,False,False,False,False
161,2011-12-16,Champanet,False,False,False,False,False,False,False,False,False,False,False,False,False
258,2012-06-29,H. beurre et plat,False,False,False,False,False,False,False,False,False,False,False,False,False
367,2013-10-29,terre à l'assiette,False,False,False,False,False,False,False,False,False,False,False,False,False
774,2016-04-28,Créme,False,False,False,False,False,False,False,False,False,False,False,False,False
866,2017-01-02,Pique-nique randonneur,False,False,False,False,False,False,False,False,False,False,False,False,False
963,2017-08-14,Pique-nique randonneur,False,False,False,False,False,False,False,False,False,False,False,False,False
1133,2018-09-28,Carré bio°,False,False,False,False,False,False,False,False,False,False,False,False,False
1337,2011-02-08,blanc à la neige,False,False,False,False,False,False,False,False,False,False,False,False,False
1427,2011-10-10,h beurre h plats,False,False,False,False,False,False,False,False,False,False,False,False,False


# Ajout du tag bio


In [30]:
df2["bio"] = df["Plat"].str.contains("bio")

<ipython-input-30-bf054ebd1671>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["bio"] = df["Plat"].str.contains("bio")


In [31]:
df2[df2["bio"] == True].head(5)

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,sauce,sucrerie,viande blanche,viande rouge,status,bio
2,2011-01-11,Tomme bio,False,False,False,True,False,False,False,False,False,False,False,False,True,True
5,2011-01-13,carottes râpées bio,False,False,False,True,True,False,False,False,False,False,False,False,True,True
39,2011-03-15,Choux blancs bio à la mimolette,False,False,False,True,True,False,False,False,False,False,False,False,True,True
41,2011-03-17,Tomme bio,False,False,False,True,False,False,False,False,False,False,False,False,True,True
70,2011-05-20,Carottes râpées bio,False,False,False,True,True,False,False,False,False,False,False,False,True,True


# Des booléens à la pondération des aliments par plats
Ici on ne veut tenir compte des catégories uniquement hors bio, produits industriels et plats préparé


In [32]:
excludingCategories = ['bio', 'plat préparé', 'produit industriel']
interestingCategories = [x for x in categories if x not in excludingCategories]
interestingCategories

['autre protéine',
 'feculent',
 'fruit',
 'laitage',
 'legume',
 'poisson fruit de mer',
 'sauce',
 'sucrerie',
 'viande blanche',
 'viande rouge']

In [33]:
def getWeight(row, category):
  if row[category] == False:
    return 0
  else: 
    # count how many categories are True
    categories = row[interestingCategories]
    count = categories.sum()
    if (count != 0):
      return 1 / count
    else:
      print("pb")
      print(row['Plat'])
      return 0

In [34]:
dfToWeight = df2[df2['status'] == True]
dfToWeight.head()

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,sauce,sucrerie,viande blanche,viande rouge,status,bio
0,2011-01-04,Crêpe au fromage,False,True,False,True,False,True,False,False,False,False,False,False,True,False
1,2011-01-04,haricots beurre / haricots plats,False,True,False,False,True,False,False,False,False,False,False,False,True,False
2,2011-01-11,Tomme bio,False,False,False,True,False,False,False,False,False,False,False,False,True,True
3,2011-01-12,Jambon fumé,False,False,False,False,False,False,False,False,False,False,True,False,True,False
4,2011-01-12,St Nectaire,False,False,False,True,False,False,False,False,False,False,False,False,True,False


In [35]:
for category in interestingCategories:
  print(category)
  dfToWeight['W'+category] = dfToWeight.apply(
      lambda row: getWeight(row, category),
      axis=1
  )

autre protéine
feculent


<ipython-input-35-60049ab9db76>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfToWeight['W'+category] = dfToWeight.apply(


fruit
laitage
legume
poisson fruit de mer
sauce
sucrerie
viande blanche
viande rouge


In [36]:
dfToWeight.head()

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,...,Wautre protéine,Wfeculent,Wfruit,Wlaitage,Wlegume,Wpoisson fruit de mer,Wsauce,Wsucrerie,Wviande blanche,Wviande rouge
0,2011-01-04,Crêpe au fromage,False,True,False,True,False,True,False,False,...,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0
1,2011-01-04,haricots beurre / haricots plats,False,True,False,False,True,False,False,False,...,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
2,2011-01-11,Tomme bio,False,False,False,True,False,False,False,False,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2011-01-12,Jambon fumé,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2011-01-12,St Nectaire,False,False,False,True,False,False,False,False,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
dfToWeight[dfToWeight['Plat'] == 'Lasagnes'].head(1)

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,...,Wautre protéine,Wfeculent,Wfruit,Wlaitage,Wlegume,Wpoisson fruit de mer,Wsauce,Wsucrerie,Wviande blanche,Wviande rouge
421,2014-02-24,Lasagnes,False,True,False,False,True,True,False,False,...,0.0,0.333333,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333


In [38]:
dfToWeight[dfToWeight['Date'] == '2019-05-22']

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,...,Wautre protéine,Wfeculent,Wfruit,Wlaitage,Wlegume,Wpoisson fruit de mer,Wsauce,Wsucrerie,Wviande blanche,Wviande rouge
2464,2019-05-22,Radis,False,False,False,False,True,False,False,False,...,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,0.000000,0.0
2465,2019-05-22,Sauce bolognaise végétale,False,False,False,False,True,True,False,True,...,0.0,0.000000,0.0,0.000000,0.5,0.0,0.5,0.0,0.000000,0.0
2466,2019-05-22,Gruyère râpé,False,False,False,True,False,False,False,False,...,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0
3643,2019-05-22,Île flottante,False,False,False,True,False,True,False,False,...,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.000000,0.0
9887,2019-05-22,Pâtes bio et gruyère râpé,False,True,False,True,False,False,False,False,...,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0


# Bio et industriel : de True à 1 pour faciliter le comptage

In [39]:
dfToWeight["Wbio"] = dfToWeight["bio"].astype(int)
dfToWeight["Windustriel"] = dfToWeight["produit industriel"].astype(int)

<ipython-input-39-4ec54ed082a2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfToWeight["Wbio"] = dfToWeight["bio"].astype(int)
<ipython-input-39-4ec54ed082a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfToWeight["Windustriel"] = dfToWeight["produit industriel"].astype(int)


In [40]:
dfToWeight[dfToWeight["Wbio"] == 1].head(2)

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,...,Wfruit,Wlaitage,Wlegume,Wpoisson fruit de mer,Wsauce,Wsucrerie,Wviande blanche,Wviande rouge,Wbio,Windustriel
2,2011-01-11,Tomme bio,False,False,False,True,False,False,False,False,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
5,2011-01-13,carottes râpées bio,False,False,False,True,True,False,False,False,...,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,1,0


à la fois bio et industriel ?

In [41]:
dfToWeight[(dfToWeight["Wbio"] == 1) & (dfToWeight["Windustriel"] == 1)]['Plat'].tolist()

['Pâté de foie bio/cornichons',
 'Compote bio',
 'Marmelade de pommes bio',
 'Crème dessert chocolat bio',
 'Pâté bio',
 'Salade de lentilles bio (Lentilles bio,maïs, surimi)',
 'Flan de poireaux / carottes bio',
 'Compote bio Manger bio',
 'Crème dessert chocolat bio°',
 'Ratatouille/blé bio',
 'Crème dessert chocolat bio°',
 'Céréales gourmandes bio',
 'Oeuf dur bio° / mayonnaise',
 'Ratatouille/ ble bio  maison',
 'Falafels bio aux pois chiches et patate douce sauce tomate',
 'Oeuf dur bio + sauce bechamel en barquette 12',
 'Pâté bio',
 'Crème chocolat bio Péard',
 'Crème dessert chocolat bio Peard',
 'Crème dessert chocolat bio',
 'Crème chocolat bio',
 'Crêpe /confiture muroise bio',
 'Mousse de foie bio',
 'Flan de courgettes bio°',
 'Bâtonnets de carottes bio° sauce crème',
 'Céréales gourmandes bio',
 'Pâté bio°',
 'Creme dessert caramel bio',
 'Ratatouille/blé bio',
 'Ratatouille/blé bio',
 'Céréales gourmandes bio',
 'pâtes tortis bio',
 'Pâté bio',
 'Pâté bio+pâté de volail

# Concaténation de plusieurs plats en une journée

Pour un jour : somme d'une catégorie divisé par # plat pour ce jour

In [42]:
excludingCategories3 = categories
interestingCategories3 = [x for x in dfToWeight.columns if x not in categories and x not in ["plat préparé", "bio", "produit industriel", "status"]]
interestingCategories3

['Date',
 'Plat',
 'Wautre protéine',
 'Wfeculent',
 'Wfruit',
 'Wlaitage',
 'Wlegume',
 'Wpoisson fruit de mer',
 'Wsauce',
 'Wsucrerie',
 'Wviande blanche',
 'Wviande rouge',
 'Wbio',
 'Windustriel']

In [43]:
dfToWeight.head()

,Date,Plat,autre protéine,feculent,fruit,laitage,legume,plat préparé,poisson fruit de mer,produit industriel,...,Wfruit,Wlaitage,Wlegume,Wpoisson fruit de mer,Wsauce,Wsucrerie,Wviande blanche,Wviande rouge,Wbio,Windustriel
0,2011-01-04,Crêpe au fromage,False,True,False,True,False,True,False,False,...,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,2011-01-04,haricots beurre / haricots plats,False,True,False,False,True,False,False,False,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0,0
2,2011-01-11,Tomme bio,False,False,False,True,False,False,False,False,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,2011-01-12,Jambon fumé,False,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
4,2011-01-12,St Nectaire,False,False,False,True,False,False,False,False,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [44]:
dfToWeight.to_csv('./output/plat_quali.csv', index = False, header=True)

# Calcul pour chaque type d'aliment son % total pour la journée en cours

In [45]:
dfToWeight = dfToWeight[interestingCategories3]
dfToWeight['count'] = dfToWeight['Plat']  # copy column for counting
dfToPercent = dfToWeight.groupby('Date').agg({
    'Wautre protéine'	: np.sum, 
    'Wfeculent'	: np.sum, 
    'Wfruit'	: np.sum, 
    'Wlaitage'	: np.sum,  
    'Wlegume'	: np.sum, 
    'Wsucrerie'	: np.sum,
    'Wsauce'	: np.sum,
    'Wpoisson fruit de mer'	: np.sum, 
    'Wviande blanche'	: np.sum, 
    'Wviande rouge': np.sum,
    'Wbio': np.sum,
    'Windustriel': np.sum,
    'count' : np.size
}).reset_index()

for x in interestingCategories3[2:]:
  dfToPercent[x] /= dfToPercent['count']

dfToPercent = dfToPercent.drop(["count"], axis=1)

dfToPercent

,Date,Wautre protéine,Wfeculent,Wfruit,Wlaitage,Wlegume,Wsucrerie,Wsauce,Wpoisson fruit de mer,Wviande blanche,Wviande rouge,Wbio,Windustriel
0,2011-01-03,0.000,0.250,0.25,0.000,0.250,0.125,0.0,0.00,0.0,0.125,0.00,0.50
1,2011-01-04,0.000,0.200,0.20,0.300,0.300,0.000,0.0,0.00,0.0,0.000,0.00,0.00
2,2011-01-05,0.000,0.200,0.10,0.200,0.300,0.000,0.0,0.00,0.2,0.000,0.00,0.00
3,2011-01-06,0.000,0.000,0.00,0.200,0.300,0.000,0.0,0.30,0.2,0.000,0.20,0.00
4,2011-01-07,0.000,0.125,0.00,0.375,0.375,0.125,0.0,0.00,0.0,0.000,0.00,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2046,2020-01-13,0.200,0.400,0.20,0.200,0.000,0.000,0.0,0.00,0.0,0.000,0.60,0.40
2047,2020-01-14,0.000,0.250,0.25,0.125,0.375,0.000,0.0,0.00,0.0,0.000,0.00,0.00
2048,2020-01-15,0.000,0.000,0.25,0.250,0.250,0.000,0.0,0.25,0.0,0.000,0.00,0.25
2049,2020-01-16,0.125,0.250,0.00,0.375,0.125,0.000,0.0,0.00,0.0,0.125,0.25,0.25


In [46]:
dfToWeight[dfToWeight['Date'] == '2020-01-16']

,Date,Plat,Wautre protéine,Wfeculent,Wfruit,Wlaitage,Wlegume,Wpoisson fruit de mer,Wsauce,Wsucrerie,Wviande blanche,Wviande rouge,Wbio,Windustriel,count
3728,2020-01-16,Choux blancs bio à la mimolette,0.0,0.0,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,1,0,Choux blancs bio à la mimolette
7488,2020-01-16,Sauté de boeuf* bourguignon,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0,0,Sauté de boeuf* bourguignon
8758,2020-01-16,Frites,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,Frites
9988,2020-01-16,Yaourt aromatisé,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,Yaourt aromatisé


In [67]:
dfToPercent = dfToPercent.rename(columns={
    'Wautre protéine': 'autre_proteine',
    'Wfeculent': 'feculent',
    'Wfruit': 'fruit',
    'Wlaitage': 'laitage',
    'Wlegume': 'legume',
    'Wpoisson fruit de mer': 'poisson_fruit_mer',
    'Wviande blanche': 'viande_blanche',
    'Wviande rouge': 'viande_rouge',
    'Wsucrerie' : 'sucrerie',
    'Wsauce' : 'sauce',
    'Wbio' : 'bio',
    'Windustriel' : 'industriel',
})

In [68]:
dfToPercent.to_csv('./output/menu_categories_percent.csv', index = False, header=True)

In [69]:
dataColumns = [x for x in dfToPercent.columns if x not in ["Date"]]
dataColumns

['autre_proteine',
 'feculent',
 'fruit',
 'laitage',
 'legume',
 'sucrerie',
 'sauce',
 'poisson_fruit_mer',
 'viande_blanche',
 'viande_rouge',
 'bio',
 'industriel']

In [70]:
dfToPercent_melt = dfToPercent.melt(id_vars =['Date'], value_vars = dataColumns, var_name ='category', value_name ='percent')
dfToPercent_melt.head()

,Date,category,percent
0,2011-01-03,autre_proteine,0.0
1,2011-01-04,autre_proteine,0.0
2,2011-01-05,autre_proteine,0.0
3,2011-01-06,autre_proteine,0.0
4,2011-01-07,autre_proteine,0.0


In [71]:
dfToPercent_melt.to_csv('./output/menu_categories_percent_melt.csv', index = False, header=True)

# Ajout des trucs de saison

In [52]:
seasons = pd.read_csv("./input/saisons.csv", sep=",")
seasons['aliment_lower'] = seasons.apply(
    lambda row: row['aliment'].lower().replace("é", "e").replace("è", "e"),
    axis=1
)
seasons.head()

,aliment,type,mois,int_mois,aliment_lower
0,pomme de terre,legume,septembre,9,pomme de terre
1,pomme de terre,legume,octobre,10,pomme de terre
2,pomme de terre,legume,novembre,11,pomme de terre
3,pomme de terre,legume,decembre,12,pomme de terre
4,pomme de terre,legume,janvier,1,pomme de terre


In [53]:
season_list = seasons['aliment_lower'].unique()

In [54]:
# get type from aliment
def get_type(df_saison, aliment):
  if aliment in season_list:
    group = df_saison[["aliment_lower", "type"]].drop_duplicates()
    return group[group['aliment_lower'] == aliment].head(1)['type'].item()
  else:
    return ''

In [55]:
get_type(seasons, 'carotte')

'legume'

In [56]:
split = dfToWeight['Date'].str.split('-', expand=True)
dfToWeight["month"] = split[1]
dfToWeight["month"] = dfToWeight["month"].replace(to_replace=r'^0', value='', regex=True)
dfSeasons = dfToWeight.loc[(dfToWeight['Wfeculent'] > 0) | (dfToWeight['Wfruit'] > 0) | (dfToWeight['Wlegume'] > 0)].reset_index()
dfSeasons = dfSeasons.drop(["count", "Wautre protéine", "Wlaitage", "Wpoisson fruit de mer", "Wviande blanche",	"Wviande rouge",	"Wsucrerie", "Wsauce"], axis=1)
dfSeasons.head(10)

,index,Date,Plat,Wfeculent,Wfruit,Wlegume,Wbio,Windustriel,month
0,0,2011-01-04,Crêpe au fromage,0.5,0.0,0.0,0,0,1
1,1,2011-01-04,haricots beurre / haricots plats,0.5,0.0,0.5,0,0,1
2,5,2011-01-13,carottes râpées bio,0.0,0.0,0.5,1,0,1
3,6,2011-01-14,Pomme,0.0,1.0,0.0,0,0,1
4,8,2011-01-18,Choux rouges,0.0,0.0,1.0,0,0,1
5,11,2011-01-19,Betteraves au chèvre,0.0,0.0,0.5,0,0,1
6,12,2011-01-19,Kiwi,0.0,1.0,0.0,0,0,1
7,13,2011-01-20,petits pois à l'anglaise,0.0,0.0,1.0,0,0,1
8,15,2011-01-21,riz pilaf,1.0,0.0,0.0,0,0,1
9,17,2011-01-26,Clémentines,0.0,1.0,0.0,0,0,1


Correction pommes de terre

In [57]:
pdt_list = ['pommes potatoes', 'pommes rissolées', 'pommes de terre', 'pommes sautées', 'Pomme noisette', 'pommes noisettes', 'pommes vapeur', 'pomme de terre', 'pomme noisettes', 'pommes noisette', 'pomme vapeur', 'pommes duchesse', 'pommes campagnardes', 'pommes parisienne']
pdt_name = 'pomme de terre'
pdt_month = seasons[seasons['aliment'] == pdt_name]['int_mois'].unique()

In [58]:
for index, row in dfSeasons.iterrows():
  match_from_list = [x for x in season_list if(x.lower() in row['Plat'].lower().replace("é", "e").replace("è", "e"))]
  res = [x for x in set(seasons.loc[seasons['int_mois'] == int(row['month'])]['aliment']) if(x.lower() in row['Plat'].lower().replace("é", "e").replace("è", "e"))]
  
  firstMatch = match_from_list[0] if len(match_from_list) > 0 else ''

  # correction pomme de terre
  if (firstMatch == 'pomme'):
    if any(pdt_item in dfSeasons.loc[index,'Plat'].lower() for pdt_item in pdt_list):
      # update name
      firstMatch = pdt_name
      #update saison
      res = ['saison'] if (int(row['month']) in pdt_month) else []
  
  dfSeasons.loc[index,'match'] = firstMatch
  dfSeasons.loc[index,'category'] = get_type(seasons, firstMatch)
  dfSeasons.loc[index,'is_saison'] = len(res) > 0

dfSeasons.head(10)


,index,Date,Plat,Wfeculent,Wfruit,Wlegume,Wbio,Windustriel,month,match,category,is_saison
0,0,2011-01-04,Crêpe au fromage,0.5,0.0,0.0,0,0,1,,,False
1,1,2011-01-04,haricots beurre / haricots plats,0.5,0.0,0.5,0,0,1,,,False
2,5,2011-01-13,carottes râpées bio,0.0,0.0,0.5,1,0,1,carotte,legume,True
3,6,2011-01-14,Pomme,0.0,1.0,0.0,0,0,1,pomme,fruit,True
4,8,2011-01-18,Choux rouges,0.0,0.0,1.0,0,0,1,chou,legume,True
5,11,2011-01-19,Betteraves au chèvre,0.0,0.0,0.5,0,0,1,betterave,legume,True
6,12,2011-01-19,Kiwi,0.0,1.0,0.0,0,0,1,kiwi,fruit,True
7,13,2011-01-20,petits pois à l'anglaise,0.0,0.0,1.0,0,0,1,pois,feculent,False
8,15,2011-01-21,riz pilaf,1.0,0.0,0.0,0,0,1,riz,feculent,False
9,17,2011-01-26,Clémentines,0.0,1.0,0.0,0,0,1,clementine,fruit,False


In [59]:
dfSeasons[dfSeasons['match'] == 'pomme']['Plat'].value_counts().index

Index(['Pomme bio°', 'Pomme', 'Marmelade de pommes', 'Tarte aux pommes',
       'Pomme bio',
       'Salade de fruits frais (ananas, pastèque, melon, pomme, ora)',
       'Tarte normande aux pommes', 'Tarte grillée aux pommes',
       'Beignet pomme', 'Compote pomme et framboise', 'Marmelade de pomme',
       'Beignet aux pommes', 'Compote de pommes / biscuit',
       'marmelade de pommes', 'Compote de pomme', 'tarte pomme speculoos',
       'pomme', 'des de pommes', 'Purée de pommes bio°', 'tarte aux pommes',
       'Pomme cuite à la framboise', 'Jus de pommes',
       'Compote pomme-framboise', 'Compote pomme framboise',
       'Marmelade de pommes bio', 'tarte pommes',
       'Compote Pomme-Abricot Bio C&Alice', 'tarte grillée aux pommes',
       'Marmelade de pommes (mettre en barquettes pour les CL ??)',
       'Compote pomme - framboise', 'Marmelade de pomme (livrée direct école)',
       'Beignets aux pommes', 'Pomme Bio', 'Tarte pomme/rhubarbe',
       'Marmelade de pommes ( en

In [60]:
dfSeasons[dfSeasons['match'] == pdt_name].head(5)

,index,Date,Plat,Wfeculent,Wfruit,Wlegume,Wbio,Windustriel,month,match,category,is_saison
46,81,2011-06-16,pomme de terre rissolées,0.25,0.25,0.25,0,0,6,pomme de terre,legume,True
52,92,2011-07-15,Pommes rissolées,0.00,0.50,0.00,0,0,7,pomme de terre,legume,False
66,117,2011-09-16,Pommes rissolées,0.00,0.50,0.00,0,0,9,pomme de terre,legume,True
107,187,2012-02-03,Pommes de terre sautées agro,0.00,1.00,0.00,0,0,2,pomme de terre,legume,True
149,269,2012-07-20,pomme vapeur,0.00,1.00,0.00,0,0,7,pomme de terre,legume,False


Quel % de match avons-nous ?

In [61]:
saison_has_match = dfSeasons[dfSeasons['match'] != '']
len(saison_has_match.index) / len(dfSeasons.index) * 100

67.44226731980406

Rajouter la saison associée à ces plat si on veut matcher plus :
Peut-être que certains ne sont pas applicable !

In [62]:
dfSeasons[(dfSeasons['match'] == '') & (dfSeasons['Wfeculent'] == 0)]['Plat'].value_counts().nlargest(20).index

Index(['Banane', 'Ananas', 'Purée de légumes verts', 'Ratatouille',
       'Yaourt aux fruits mixés', 'Salade coleslaw', 'Gaspacho',
       'Gratinée de légumes', 'Compote', 'Poêlée de légumes',
       'Sauce bolognaise végétale', 'Galettes de légumes', 'Potage de légumes',
       'Salade au surimi', 'banane', 'Fruit', 'Salade',
       'Fromage blanc / Coulis de fruits rouges', 'Salade au thon',
       'Salade de fruits frais'],
      dtype='object')

Qui est-ce qu'on mange mais qui n'est pas de saison ?

In [63]:
not_saison = dfSeasons[(dfSeasons['is_saison'] == False) & (dfSeasons['match'] != '')]
not_saison.head(15)

,index,Date,Plat,Wfeculent,Wfruit,Wlegume,Wbio,Windustriel,month,match,category,is_saison
7,13,2011-01-20,petits pois à l'anglaise,0.0,0.0,1.0,0,0,1,pois,feculent,False
8,15,2011-01-21,riz pilaf,1.0,0.0,0.0,0,0,1,riz,feculent,False
9,17,2011-01-26,Clémentines,0.0,1.0,0.0,0,0,1,clementine,fruit,False
21,38,2011-03-14,Mâche,0.0,0.0,1.0,0,0,3,mâche,legume,False
25,44,2011-03-21,Abricots secs,0.0,1.0,0.0,0,0,3,abricot,fruit,False
26,46,2011-03-23,Purée de céleri,0.0,0.0,1.0,0,0,3,celeri,legume,False
30,54,2011-04-12,Haricots verts,0.5,0.0,0.5,0,0,4,haricots verts,legume,False
31,55,2011-04-12,Orange,0.0,1.0,0.0,0,0,4,orange,fruit,False
32,56,2011-04-13,panaché de choux blancs et rouges,0.0,0.0,1.0,0,0,4,chou,legume,False
35,61,2011-04-26,petits pois,0.0,0.0,1.0,0,0,4,pois,feculent,False


Quel pourcentage est hors saison (parmis les aliments reconnu dans liste de saisonalité ?

In [64]:
len(not_saison.index) / len(saison_has_match.index) * 100

30.66147859922179

In [65]:
dfSeasons.to_csv('./output/menu_de_saisons.csv', index = False, header=True)

In [66]:
dfSeasons['is_saison'].value_counts()

False    3043
True     2673
Name: is_saison, dtype: int64